In [ ]:
import os
import pickle
import cv2 as cv
import numpy as np

from time import time

Clear duplicates from annotations 

In [2]:
for i in range(6804):
    with (open(f"ann/obj_train_data/frame_{str(i).zfill(6)}.txt") as f1, 
          open(f"ann/labels/frame_{str(i).zfill(6)}.txt", "w") as f2):
        f2.writelines(set(f1.readlines()))

Load fiftyone for dataset visualisation

In [ ]:
import fiftyone as fo

# fo.utils.video.reencode_video("/vids/IMG_1855.MP4", "/vids/NEW_IMG_1855.MP4")

# Create a dataset from a list of videos
dataset = fo.Dataset.from_videos(
    ["./vids/IMG_1855.MP4"]
)
    
session = fo.launch_app(dataset)

session.wait()

Get images for dataset with big mathematical distance between them

In [ ]:
DUPLICATES_MAX_DIFF = 1e7
FRAMES_STEP = 250
to_skip = set()
timer = time()
to_skip_prev = len(to_skip)
for dirname in dirs:
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue

    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    frames = sorted(os.listdir(frames_path))
    for i in range(len(frames) // FRAMES_STEP + 1):
        ref_frame = frames[i*FRAMES_STEP]
        similar_to_ref = set()
        left_border = i*FRAMES_STEP + 1
        right_border = min((i+1)*FRAMES_STEP, len(frames))
        img1 = cv.imread(f'{frames_path}/{ref_frame}', cv.IMREAD_GRAYSCALE)
        for j in range(left_border, right_border):
            img2 = cv.imread(f'{frames_path}/{frames[j]}', cv.IMREAD_GRAYSCALE)
            if np.sum(cv.absdiff(img1, img2)) <= DUPLICATES_MAX_DIFF:
                similar_to_ref.add(f'{frames_path}/{frames[j]}')
        to_skip.update(similar_to_ref)

    print(f'{dirname} - {int(time() - timer)}s - {len(to_skip) - to_skip_prev} to skip')
    to_skip_prev = len(to_skip)

Check `to_skip` set

In [ ]:
len(to_skip)

In [ ]:
with open('to_skip.p', 'wb') as f:
    pickle.dump(to_skip, f)

with open('to_skip.p', 'rb') as f:
    to_skip_p = pickle.load(f)

Move images for train and val

In [ ]:


RATIO = 0.8
VID_LEN = 15002
frames_amount = 12*VID_LEN - len(to_skip)
counter = 0
counter_prev = 0
train_frames_path = f'{DS_FINAL_DIRNAME}/{TRAIN_DIRNAME}/{FRAMES_DIRNAME}'
train_labels_path = f'{DS_FINAL_DIRNAME}/{TRAIN_DIRNAME}/{LABELS_DIRNAME}'
val_frames_path = f'{DS_FINAL_DIRNAME}/{VAL_DIRNAME}/{FRAMES_DIRNAME}'
val_labels_path = f'{DS_FINAL_DIRNAME}/{VAL_DIRNAME}/{LABELS_DIRNAME}'
for dirname in [train_frames_path, train_labels_path, val_frames_path, val_labels_path]:
    os.makedirs(dirname, exist_ok=True)
for dirname in dirs:
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue

    labels_path = f'{DS_DIRNAME}/{dirname}/{LABELS_DIRNAME}'
    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    for frame_name in os.listdir(frames_path):
        if frame_name.startswith(dirname):
            label_name = frame_name[:-4] + '.txt'
            if counter <= frames_amount * RATIO:
                os.rename(
                    f'{frames_path}/{frame_name}',
                    f'{train_frames_path}/{frame_name}'
                )
                os.rename(
                    f'{labels_path}/{label_name}',
                    f'{train_labels_path}/{label_name}'
                )
            else:
                os.rename(
                    f'{frames_path}/{frame_name}',
                    f'{val_frames_path}/{frame_name}'
                )
                os.rename(
                    f'{labels_path}/{label_name}',
                    f'{val_labels_path}/{label_name}'
                )
            counter += 1

    train_or_val = 'train' if counter <= frames_amount * RATIO else 'val'
    print(f'{dirname} - {counter - counter_prev} - {train_or_val}')
    counter_prev = counter
